In [131]:
import numpy as np
import pandas as pd 
from sklearn.base import BaseEstimator
import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

%matplotlib inline

In [164]:
class SGD_log_reg(BaseEstimator):
    
    
    def __init__(self, eta=1e-3, n_epochs=3):
        self.eta = eta
        self.n_epochs = n_epochs
        self.weights_ = []
        self.accuracy_ = []
    
    
    def fit(self, X, y):
        X = np.hstack([np.ones([X.shape[0], 1]), X])
        w = np.zeros(X.shape[1])
        
        for i in range(self.n_epochs): 
            for j in range(X.shape[0]):
                temp_exp = np.exp(-y[j] * w.dot(X[j]))
                new_weights = w.copy()
                new_weights[0] += -(self.eta * y[j] * temp_exp / (1 + temp_exp))
                for k in range(1, len(w)):
                    new_weights[k] += -(self.eta * y[j] * X[j][k] * temp_exp / (1 + temp_exp))
                w = new_weights.copy()
                self.weights_.append(w)
                self.accuracy_.append(accuracy_score(y, custom_predict(X, w)))

        
        self.w_ = self.weights_[np.argmax(self.accuracy_)]
        return self
    
    
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))


    def custom_predict(X, w):
        res = []
        for i in range(len(X)):
            sig = sigmoid(w.dot(X[i]))
            res.append(1 if sig >= 0 else -1)
        return res

    
    def predict(self, X):
        X = np.hstack([np.ones([X.shape[0], 1]), X])
        return custom_predict(X, self.w_)
                


In [42]:
# def fill_nan(table):
#     for col in table.columns:
#         table[col] = table[col].fillna(table[col].median())
#     return table

# DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"
# data = pd.read_csv(DATA_PATH + "credit_scoring_sample.csv", sep=";")
# independent_columns_names = [x for x in data if x != "SeriousDlqin2yrs"]
# table = fill_nan(data)
# X = table[independent_columns_names]
# y = table["SeriousDlqin2yrs"]

In [67]:
data = pd.read_csv("car_data.csv")
data["Gender"] = np.where(data["Gender"] == "female", 0, 1)
data = data.drop("User ID", axis=1)
y = data["Purchased"]
X = data.drop("Purchased", axis=1)
data.head()

,Gender,Age,AnnualSalary,Purchased
0,1,35,20000,-1
1,1,40,43500,-1
2,1,49,74000,-1
3,1,40,107500,1
4,1,25,79000,-1


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [142]:
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=101)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train = list(y_train)
y_test = list(y_test)

In [161]:
my_sgd_logreg = SGD_log_reg(eta=0.1, n_epochs=10)
my_sgd_logreg.fit(X_train_scaled, y_train)

SGD_log_reg(eta=0.1, n_epochs=10)

In [162]:
preds = my_sgd_logreg.predict(X_test_scaled)

In [163]:
accuracy_score(y_test, preds)

0.62

In [95]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=0.1, random_state=5, class_weight="balanced")
lr.fit(X_train_scaled, y_train)
preds_ne_eblana = lr.predict(X_test_scaled)
accuracy_score(y_test, preds_ne_eblana)

0.8366666666666667